In [ ]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 13.3 MB/s 


In [ ]:
vocabulary_size = 603432

In [ ]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = Sequence([
            NFC()
        ])
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        trainer = BpeTrainer(vocab_size=vocabulary_size, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        self.tokenizer.train(paths, trainer)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)

In [ ]:
paths = ['cwe-train.txt']

In [ ]:
paths

['cwe-train.txt']

In [ ]:
tokenizer = BPE_token()
tokenizer.bpe_train(paths)
tokenizer.save_tokenizer('.')

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 6.3 MB/s 
     |████████████████████████████████| 163 kB 58.4 MB/s 


In [ ]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained('.')
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id,
  n_embd=256,
  n_layer=4,
  n_head=4
)
# creating the model
model = TFGPT2LMHeadModel(config)
model(model.dummy_inputs)
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 7654656   
 r)                                                              
                                                                 
Total params: 7,654,656
Trainable params: 7,654,656
Non-trainable params: 0
_________________________________________________________________


In [ ]:
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
    x = f.read()
  single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [ ]:
string_tokenized[:10]

[118, 744, 685, 164, 165, 107, 7166, 9279, 7, 8466]

In [ ]:
examples = []
block_size = 100
BATCH_SIZE = 128
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [ ]:
num_epoch = 100
history = model.fit(dataset,batch_size=1000, epochs=num_epoch)

Epoch 1/100
8/8 [==============================] - 2s 296ms/step - loss: 8.5955 - accuracy: 0.0678
Epoch 2/100
8/8 [==============================] - 2s 300ms/step - loss: 8.5192 - accuracy: 0.0680
Epoch 3/100
8/8 [==============================] - 2s 298ms/step - loss: 8.4440 - accuracy: 0.0680
Epoch 4/100
8/8 [==============================] - 2s 298ms/step - loss: 8.3693 - accuracy: 0.0694
Epoch 5/100
8/8 [==============================] - 2s 297ms/step - loss: 8.2945 - accuracy: 0.0712
Epoch 6/100
8/8 [==============================] - 2s 299ms/step - loss: 8.2215 - accuracy: 0.0734
Epoch 7/100
6/8 [=====================>........] - ETA: 0s - loss: 8.1581 - accuracy: 0.0747

KeyboardInterrupt: ignored

In [ ]:
text = "An bhfuil"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 50,
  num_beams = 5,
  temperature = 0.4,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)
print(tokenizer.decode(beam_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


<unk>n bhfuil,, na,.. na na., ", wa, yesu, wanhu,', kwa, kwaviya,?", keza, mulungu, mbali, novimanya, muna, yoyose, nhaguso, kaidi,


In [ ]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
model_to_save = model.module if hasattr(model, 'module') else model
output_model_file = os.path.join('.', WEIGHTS_NAME)
output_config_file = os.path.join('.', CONFIG_NAME)
# save model and model configs
model.save_pretrained('.')
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained('.')

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.json',
 './merges.txt',
 './added_tokens.json')

In [ ]:
# can then avoid retraining by loading these files when they exist:
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
model = TFGPT2LMHeadModel.from_pretrained(output_dir)

NameError: ignored